# 3. Bilateral filtering

In [15]:
import numpy as np
import cv2
import os
import matplotlib
from matplotlib import pyplot as plt
from numpy.lib.stride_tricks import as_strided
import time
%matplotlib inline

os.chdir("../images/")

def str_int_lex(m):
    if m<10: return "0"+str(m)
    return str(m)

Some basic dependencies before we start - for getting Image Dimensions, Color Space Transformations, Channel-Splitting

In [20]:
def getColorSpaces(image):
    rgb = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    return rgb,gray

def getImageDimension(image):
    height,width = image.shape[:2]
    
    return height,width

def showImage(image,title,cmap):
    plt.imshow(image,cmap=cmap)
    plt.axis('off')
    plt.title(title)


def splitRGBChannels(image):
    red, green, blue= cv2.split(image)
    return red, green, blue

def getNormalizedImage(image):
    norm_image=image.copy()
    norm_image = np.maximum(norm_image, np.zeros(norm_image.shape))
    norm_image = np.minimum(norm_image, 255 * np.ones(norm_image.shape))
    norm_image = norm_image.round().astype(np.uint8)
    
    return norm_image

def applyLaplacian(gray,kernel_size):
    # Apply Gaussian Blur
    sigma=1.0
      
    blur = cv2.GaussianBlur(gray,(kernel_size,kernel_size),sigma)
    
    #Apply the laplacian now
    laplacian = cv2.Laplacian(blur,cv2.CV_16S,ksize=3)
    laplacian = np.maximum(laplacian, np.zeros(laplacian.shape))
    laplacian = np.minimum(laplacian, 255 * np.ones(laplacian.shape))
    laplacian = laplacian.round().astype(np.uint8)
    
    return laplacian

def getHistogram(image, bins=256):
    
    image_pixels=image.flatten()
    # array with size of bins, set to zeros
    histogram = np.zeros(bins)
    
    # loop through pixels and sum up counts of pixels
    for pixel in image_pixels:
        histogram[pixel] += 1
    
    # return our final result
    return histogram

def applyUnsharp(image, kernel_size=(5, 5)):
    """Return a sharpened version of the image, using an unsharp mask."""
    k=1.0
    sigma=1.0
    blurred = cv2.GaussianBlur(image, kernel_size, sigma) #Lowpass
    sharpened = float(k + 1) * image - float(k) * blurred #gmask
    sharpened =getNormalizedImage(sharpened)
    return sharpened

def getResizedImage(img,scale_percent = 50 ):
    # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized    


def getPaddedImage(image,pad,channels,data_type):
    if channels==3:
        return np.pad(image, ((pad, pad), (pad, pad), (0, 0)), 'symmetric').astype(data_type)
    else:
        return np.pad(image, ((pad, pad), (pad, pad)), 'symmetric').astype(data_type)
    


In [32]:
def applyBF(img_gray,k,sigma_domain,sigma_range):
    padding=k//2
    h,w = getImageDimension(img_gray)
    gray = getPaddedImage(img_gray,padding,1,data_type='uint8')
    view_shape = tuple(np.subtract(gray.shape, (k,k)) + 1) + (k,k)
    # Create a gausian filter    
    gaussX,gaussY = np.meshgrid(np.linspace(-(padding),(padding),k),np.linspace(-(padding),(padding),k))    
    kernel_domain = np.exp(-(gaussX**2 + gaussY**2)/(2*(sigma_domain**2)))
    
    expanded_input = as_strided(gray, shape = view_shape, strides = gray.strides * 2)
    kernel_range = expanded_input -expanded_input[:,:,padding,padding][:,:,np.newaxis,np.newaxis]    
    kernel_range = np.exp(-kernel_range/(2*sigma_range**2))
    kernel = kernel_range * kernel_domain
    
    filtered_gray = np.sum(kernel*expanded_input,axis=(2,3))/np.sum(kernel,axis=(2,3))
    return filtered_gray


In [33]:
def getGaussianKernel(kernel_size=3,sigma=1.0):
    
    pad=kernel_size//2    
    ax=np.linspace(-pad,pad,kernel_size)
    ay=np.linspace(-pad,pad,kernel_size)    
    np.arange(2 * pad + 1) - pad
    X,Y = np.meshgrid(ax,ay)     
    kernel = np.exp(-(np.square(X)+np.square(Y))/(2*np.square(sigma)))

    return kernel

def crossBF(image1, image2, sigma_domain, sigma_range):
    pad_img = int(np.ceil(3 * sigma_domain))
    if image1.ndim == 3:
        channels=3
    if image1.ndim == 2:
        channels=2
    h,w=getImageDimension(image1)
    image1_padded = getPaddedImage(image1,pad_img,channels,np.float32)
    image2_padded = getPaddedImage(image2,pad_img,channels,np.int32)
    output = np.zeros_like(image1)
    # A lookup table for range kernel
    LUT = np.exp(-np.arange(256) * np.arange(256) * 1 / (2 * sigma_range**2))
    # Generate a spatial Gaussian function
    x, y = np.meshgrid(np.arange(2 * pad_img + 1) - pad_img, np.arange(2 * pad_img + 1) - pad_img)
    kernel_domain = np.exp(-(x * x + y * y) * 1 / (2 * sigma_domain**2))
    if image1_padded.ndim == 2:
        for y in range(pad_img, pad_img + h):
            for x in range(pad_img, pad_img + w):
                W = LUT[np.abs(image2_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1] - image2_padded[y, x])] * kernel_domain
                output[y - pad_img, x - pad_img] = np.sum(W * image1_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1]) / np.sum(W)
    
    if image1_padded.ndim == 3 :  
        for y in range(pad_img, pad_img + h):
            for x in range(pad_img, pad_img + w):
                W = LUT[abs(image2_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1, 0] - image2_padded[y, x, 0])] * \
                    LUT[abs(image2_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1, 1] - image2_padded[y, x, 1])] * \
                    LUT[abs(image2_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1, 2] - image2_padded[y, x, 2])] * \
                    kernel_domain
                Waccum = np.sum(W)
                output[y - pad_img, x - pad_img, 0] = np.sum(W * image1_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1, 0]) / Waccum
                output[y - pad_img, x - pad_img, 1] = np.sum(W * image1_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1, 1]) / Waccum
                output[y - pad_img, x - pad_img, 2] = np.sum(W * image1_padded[y - pad_img:y + pad_img + 1, x - pad_img:x + pad_img + 1, 2]) / Waccum
    return output


In [34]:
img1 = cv2.imread('input_data/mountain.jpg')
img2 = cv2.imread('input_data/cake_flash.jpg')
img3 = cv2.imread('input_data/cake_noflash.jpg')

img1_rgb, img1_gray = getColorSpaces(img1)
img2_rgb, img2_gray = getColorSpaces(img2)
img3_rgb, img3_gray = getColorSpaces(img3)

img1_r, img1_g, img1_b = splitRGBChannels(img1)

sigma_domain=5
sigma_range=20

img1_r_bf = applyBF(img1_r,5,sigma_domain,sigma_range)
img1_g_bf = applyBF(img1_g,5,sigma_domain,sigma_range)
img1_b_bf = applyBF(img1_b,5,sigma_domain,sigma_range)

img1_rgb_bf = np.zeros([img1_r_bf.shape[0],img1_r_bf.shape[1],3],dtype='uint8')
img1_rgb_bf[:,:,0] = img1_r_bf[:,:]
img1_rgb_bf[:,:,1] = img1_g_bf[:,:]
img1_rgb_bf[:,:,2] = img1_b_bf[:,:]

cv2.imwrite('mountain_bilateral.jpg',img1_rgb_bf)

sigma_domain=2.5
sigma_range=50

img_cbf = crossBF(img2, img3, sigma_domain, sigma_range)

cv2.imwrite('cake-out.jpg',img_cbf)

True